# semantic grep vectorspace test bench

## pending design decision 
index or ctx or ass vectors?

## theory

five levels:

all presuppose existence of reference space with ctx vectors and ass space with other ctx vectors. should those be made with lemma or surface form?

should reference space addition be weighted not by pmi or idf but by referenitality of the concept?


1. associative-discourse level: wide context window; invidivual vs class?
2. reference: instance vs class, which?; narrow context window; not all words relevant; invidivual vs class?
3. syntax+semantics (roles, case, adverbials): use lemma or surface form?
4. morph (number, definite, tense): use index vector, use lemma or surface form?
5. situ (ĺatlong time source audience channel)

clause adverbials
negation
subordination
coordination


## what to test with?

- ingest fdg and use it for first tests
- ingest propbank 


In [1]:
import re
import math
import random
import time
import nltk

In [3]:
def sparseadd(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            result[l] = onevec[l]
        for k in othvec:
            if k in result:
                result[k] = result[k]+othvec[k]*float(weight)
            else:
                result[k] = othvec[k]*float(weight)
    except:
        print("sparseadd(): error")
    return result
def sparsemultiply(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            if l in othvec:
                result[l] = onevec[l]*othvec[l]*float(weight)
    except:
        print("sparsemultiply(): error ")
    return result
def sparsexor(onevec,othvec):
    result={}
    try:
        for l in range(len(onevec)):
            if ((l in onevec) and not (l in othvec)):
                result[l] = 1
            if (not (l in onevec) and (l in othvec)):
                result[l] = 1        
    except:
        print("sparsexor(): error ")
    return result

def newrandomvector(n,k):
    vec = {}
    if (k > 0):# no need to be careful about this, right? and k % 2 == 0):
        nonzeros = random.sample(list(range(n)),k)
        negatives = random.sample(nonzeros,k//2)
        for i in nonzeros:
            vec[str(i)] = 1;
        for i in negatives:
            vec[str(i)] = -1;
    return vec

def newoperator(n):
    k = n//10
    return newrandomvector(n,k)

def sparsecosine(xvec,yvec,truncate=True):
    x2 = 0;
    y2 = 0;
    xy = 0;
    try:
        for i in xvec:
            x2 = x2+xvec[i]*xvec[i]
    except:
        print("sparsecosine(): error at position ",i)
    try:
        for j in yvec:
            y2 = y2+yvec[j]*yvec[j]
            if j in xvec:
                xy = xy+xvec[j]*yvec[j]
    except:
        print("sparsecosine(): errors at position ",j)
    if (x2*y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)*math.sqrt(y2))
    if (truncate):
        cos=int(cos*10000)/10000
    return cos

def sparselength(vec,truncate=True):
    x2 = 0
    length=0
    try:
        for i in vec:
            x2 = x2+vec[i]*vec[i]
    except:
        print("sparselength(): error at position ",i)
    if (x2 > 0):
        length = math.sqrt(x2)
    if (truncate):
        length=int(length*10000)/10000
    return length

def comb(vec,k=0.1):
    newvector={}
    n=int(k*dimensionality/2)
    sorted_items=sorted(vec.items(), key=lambda x:x[1])
    bot=sorted_items[:n]
    top=sorted_items[-n:]
    for l in bot:
        newvector[l[0]]=l[1]
    for l in top:
        newvector[l[0]]=l[1]
    return newvector

def sparsesum(vec):
    s=0
    for i in vec:
        s += float(vec[i])
    return s

def normalise(vec,truncate=True):
    newvector={}
    vlen=sparselength(vec,False)
    if (vlen > 0):
        for i in vec:
            newvector[i]=vec[i]/math.sqrt(vlen*vlen)
    else:
        newvector=vec
    return newvector

def modify(vec,factor):
    newvector={}
    for i in vec:
        if (random.random() > factor):
            newvector[i]=vec[i]
        else:
            newvector[i]=float(vec[i])*(0.5-random.random())*2.0
    return newvector

def createpermutation(k):
    permutation=random.sample(range(k), k)
    return permutation
    
def permute(vector,permutation):
    newvector={}
    try:
        for i in range(len(permutation)):
            if str(i) in vector:
                newvector[str(permutation[i])]=vector[str(i)]
    except:
        newvector=vector
        print("permute(): no permutation done, something wrong")
    return newvector
    

In [4]:
wordspacefile="/home/jussi/Desktop/data/wordspaces/gavagai-news-22-morethan5.wordspace"
dimensionality=2000
denseness=10
fdgtextfile="/home/jussi/Desktop/data/GH95-fdg/950102.sgml.fdg.xml"
unittestfile="/home/jussi/Desktop/data/sents.txt"
unittestwordspacefile="/home/jussi/Desktop/data/wordspaces/gavagai-news-22-morethan5.wordspace.unittest"

#fdgtextfile="/home/jussi/Desktop/data/mini.fdg"

definite=createpermutation(dimensionality)
indefinite=createpermutation(dimensionality)
future=createpermutation(dimensionality)
past=createpermutation(dimensionality)
present=createpermutation(dimensionality)

#definiteoperator=newoperator(dimensionality)
#indefiniteoperator=newoperator(dimensionality)

agentoperator=newoperator(dimensionality)
patientoperator=newoperator(dimensionality)
mainverboperator=newoperator(dimensionality)



In [55]:
class clause:
    cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
    def __init__(self,string):
        self.surfacestring=string
        self.cleanedutterance=re.sub(clause.cleanuppattern,"",string)
        self.tokens=self.cleanedutterance.lower().split()
        self.agent=None
        self.event=None
        self.patient=None
        self.instrument=None
        self.location=None
    def __str__(self):
        return self.surfacestring
    
class referent:
    def __init__(self, string):
        self.surfacestring=string
        self.definite=True
        self.number=1
        
class event:
    def __init__(self, string):
        self.surfacestring=string    
        self.tense=None #past present or future
        self.aspect=None #ongoing perfect or pointwise
        self.mood=None #indicative, irreal, potential, optative, not, imperative

def semanticparse(string):
    utterance=clause(string)  
    words=utterance.tokens
    utterance.agent=referent(words.pop(0))       # english grammar: first word is subject
    utterance.agent.definite=True            # for now, assume subject is definite
    mainv=words.pop(0)              # main verb is second word of utterance in english
    if (mainv=="will"):             #    sometimes an auxiliary, for now "will" is accepted
        utterance.event=event(words.pop(0))
        utterance.event.tense="FUTURE"
    else:
        utterance.event=event(mainv)
    det=words.pop(0)                # object carries a determiner 
    utterance.patient=referent(words.pop(0))
    if (det=="a"):                   # if det is "a", object is indefinite
        utterance.patient.definite=False
    else: 
        utterance.patient.definite=True       # else ("the" or "my" or some such) object is definite
    return utterance


from nltk.parse.stanford import StanfordDependencyParser
path_to_stanford_corenlp = '/usr/share/stanford-corenlp-full/'
parser_jar='stanford-corenlp.jar'
path_to_models = path_to_stanford_corenlp+'models/'
model_jar='stanford-english-corenlp-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_stanford_corenlp+parser_jar, path_to_models_jar=path_to_models+model_jar)

def semanticdepparse(string,debug=False):
    utterance=clause(string)
    utterance.agent=referent("epsilon")
    utterance.patient=referent("epsilon")
    utterance.event=event("epsilon")
    depgraph=dependency_parser.raw_parse(string)
    scratch={}
    for deps in depgraph:
#        if debug:
#            print(deps)
        for nodeindex in deps.nodes:
            if deps.nodes[nodeindex]['rel'] == 'nsubj':
                utterance.agent=referent(deps.nodes[nodeindex]['word'])
                if nodeindex in scratch: 
                    if scratch[nodeindex]=="def":
                        utterance.agent.definite=True
                    elif scratch[nodeindex]=="indef":
                        utterance.agent.definite=False
            elif deps.nodes[nodeindex]['rel'] == 'nmod:poss':
                scratch[deps.nodes[nodeindex]['head']]="def"
            elif deps.nodes[nodeindex]['rel'] == 'det':
                if (deps.nodes[nodeindex]['word']=="a" or deps.nodes[nodeindex]['word']=="an"):
                    scratch[deps.nodes[nodeindex]['head']]="indef"
                elif (deps.nodes[nodeindex]['word']=="the"):
                    scratch[deps.nodes[nodeindex]['head']]="def"
            elif deps.nodes[nodeindex]['rel'] == 'root':
                utterance.event=event(deps.nodes[nodeindex]['word'])
            elif deps.nodes[nodeindex]['rel'] == 'dobj':
                utterance.patient=referent(deps.nodes[nodeindex]['word'])
                if nodeindex in scratch: 
                    if scratch[nodeindex]=="def":
                        utterance.patient.definite=True
                    elif scratch[nodeindex]=="indef":
                        utterance.patient.definite=False                
    if debug:
            print("semanticdepparse(): e:",utterance.event.surfacestring," a:",utterance.agent.surfacestring," p:",utterance.patient.surfacestring)
    return utterance

# i feed my cat
# i will feed my dog

In [13]:
def chkwordspace(words,debug=False):
    i = 0
    for w in words:
        if w in vectorspace:
                i += 1
        else:
            if debug:
                print("chkwordspace(): ",w," is new and is now hallucinated.")
            denseness2=dimensionality//2
            vectorspace[w] = newrandomvector(dimensionality,denseness2)
            indexspace[w] = newrandomvector(dimensionality,denseness)
            associationspace[w] = newrandomvector(dimensionality,denseness2)    

def wordlikeness(w1,w2):
    return sparsecosine(vectorspace[w1],vectorspace[w2])

def dowordspacefile(wordspacefile):
    linepattern = re.compile(r'^\(\"([\-\w]+)\"\s+#S2000([\d\,\-\.\-\+;e]+)*:\s+#S2000([\d\-\+;\.e\,]+)*:\s+#S2000([\.\d\-\+;e\,]+)*:\s+(\d+)\)')
    vecpattern = re.compile(r';?(\d+)([\+\-][\,\.\d]+(e-\d\d)?)')
    alicefile=open(wordspacefile,'r')
    global vectorspace
    vectorspace={}
    global globalfrequency
    globalfrequency={}
    global indexspace
    indexspace={}
    global associationspace
    associationspace={}
    aliceline = alicefile.readline()
    a=1
    b=0
    w=0
    while aliceline:
        a = a+1
        m=linepattern.match(aliceline)
        if m:
            token=m.groups()[0]
            ind=m.groups()[1]
            doc=m.groups()[2]
            ctx=m.groups()[3]
            freq=int(m.groups()[4])
            try:
                sparsevec={}
                cells = re.findall(vecpattern,ctx)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                vectorspace[token] = sparsevec
                sparsevec={}
                cells = re.findall(vecpattern,ind)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                indexspace[token] = sparsevec
                globalfrequency[token] = freq
                sparsevec={}
                w=w+1
            except:
                print("fail: ",token," ",a)
        else:
            b=b+1
        aliceline = alicefile.readline()
    print(a," lines seen; ",w," items added ",b," items disregarded")
    alicefile.close()
    


In [41]:
dowordspacefile(unittestwordspacefile)
chkwordspace(["epsilon"],True)

194  lines seen;  193  items added  0  items disregarded


In [14]:
utterancespace = {}
dependencyanalysisstore={}
def initunittest(debug=False):
    with open(unittestfile,"r") as sents:
        s = sents.readline().rstrip()
        i = 0
        while s:
            print(s)
            wds=nltk.word_tokenize(s.lower())
            chkwordspace(wds,debug)
            try:
                c = semanticdepparse(s.lower(),debug)
                dependencyanalysisstore[s]=c
                u = utterancevector(c,debug)
                utterancespace[s] = u
                i += 1
            except:
                print("****",s)
            s = sents.readline()
def rununittest():
    for v in utterancespace:
        print(v)
        for w in utterancespace:
            print("          ",sparsecosine(utterancespace[v],utterancespace[w])," ",w)

    
    

In [74]:
def showtree(sentence):
    depgraph=dependency_parser.raw_parse(sentence.lower())
    for deps in depgraph:
        print(deps)
        for nodeindex in deps.nodes:
            print(deps.nodes[nodeindex])


In [77]:
showtree("I never have a bagel.")

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7fc8cb258ae8>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [3]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'FW',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 3,
                 'lemma': '_',
                 'rel': 'nsubj',
                 'tag': 'FW',
                 'word': 'i'},
             2: {'address': 2,
                 'ctag': 'RB',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 3,
                 'lemma': '_',
                 'rel': 'neg',
                 'tag': 'RB',
                 'wo

In [20]:
def utterancevector(utterance,debug=False,lexical=True,morphology=True,syntacticroles=True,morphologyrole=False):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    if debug:
        debugvec={}
    if lexical:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.patient.surfacestring])))#,sparselength(vectorspace[utterance.patient]))
        if (utterance.event):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.event.surfacestring])))#,sparselength(vectorspace[utterance.verb]))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.agent.surfacestring])))#,sparselength(vectorspace[utterance.agent]))
    if debug:
        print("utterancevec() lexicon: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if morphology:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],present)) )  
    if debug:
        print("utterancevec() morphology: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if morphologyrole:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(patientoperator,definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(patientoperator,indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,present)) )  
    if debug:
        print("utterancevec() morphologyrole: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    if syntacticroles:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,sparsemultiply(patientoperator,normalise(indexspace[utterance.patient.surfacestring])))
        if (utterance.event):
            uttvec=sparseadd(uttvec,sparsemultiply(mainverboperator,normalise(indexspace[utterance.event.surfacestring])))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,sparsemultiply(agentoperator,normalise(indexspace[utterance.agent.surfacestring])))
    if debug:
        print("utterancevec() syntacticrole: ",sparsecosine(uttvec,debugvec))
        debugvec=uttvec
    return uttvec



In [71]:
def tripletest(s1,s2,s3,debug=False):
    t1=semanticdepparse(s1.lower())
    t2=semanticdepparse(s2.lower())
    t3=semanticdepparse(s3.lower())
    for m in [False,True]:
        u1=utterancevector(t1,debug,True,m,False,False)
        u2=utterancevector(t2,debug,True,m,False,False)
        u3=utterancevector(t3,debug,True,m,False,False)
        c12=sparsecosine(u1,u2)
        c13=sparsecosine(u1,u3)
        c23=sparsecosine(u2,u3)
        print("1.0000\t{}\t{}".format(c12,c13))
        print("......\t1.0000\t{}".format(c23))
        print("......\t......\t1.0000")
        print("----------------------")

In [72]:
tripletest("The cat drinks the milk","A cat drank my milk","My brother sails a boat")

1.0000	0.2545	0.8043
......	1.0000	0.006
......	......	1.0000
-----------------------
1.0000	0.111	0.8034
......	1.0000	-0.0184
......	......	1.0000
-----------------------


In [31]:
initunittest(True)

I want to visit my uncle on the farm.
semanticdepparse(): e: want  a: i  p: uncle
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7038
utterancevec() morphologyrole:  0.9999
utterancevec() syntacticrole:  0.9836
We can not get there by train.

semanticdepparse(): e: get  a: we  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.709
utterancevec() morphologyrole:  0.9999
utterancevec() syntacticrole:  0.9833
I usually take the bus.

semanticdepparse(): e: take  a: i  p: bus
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.6943
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.9835
It is too far to walk.

semanticdepparse(): e: far  a: it  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7211
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.9835
I have never walked that far.

semanticdepparse(): e: walked  a: i  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.6998
utt

semanticdepparse(): e: ran  a: boat  p: aground
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.6944
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.9605
The boat was heading from Latvia to Sweden when it sank near Copenhagen 

semanticdepparse(): e: heading  a: it  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7379
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.9736
The boat ran aground on a wreck on its way back 

semanticdepparse(): e: ran  a: boat  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.6942
utterancevec() morphologyrole:  1.0
utterancevec() syntacticrole:  0.9756
The yacht ran aground on the North coast of Caithness 

semanticdepparse(): e: ran  a: yacht  p: epsilon
utterancevec() lexicon:  0.0
utterancevec() morphology:  0.7076
utterancevec() morphologyrole:  0.9999
utterancevec() syntacticrole:  0.9767


In [32]:
rununittest()

The wind drove two boats aground off Santa Island 

           1.0   The wind drove two boats aground off Santa Island 

           -0.0286   Riding in the rain is no fun.

           -0.0175   A yacht is expensive

           0.0624   It is too far to walk.

           0.0305   A boat skipper let a barge run aground in January off Puerto Rico 

           0.165   A Navy patrol boat went aground on an uncharted sandbar 

           -0.0057   Planes are faster than trains.

           0.0112   I would like a sailboat

           -0.013   A sloop is affordable

           0.0108   I already know how to ride the bicycle.

           0.2124   The boat ran aground on a wreck on its way back 

           0.0638   Seven crew abandoned the boat when it struck a rock 

           0.0433   In fourteen hundred and ninety two, Columbus sailed the ocean blue.

           -0.031   Powerboats are more fun than sailboats

           0.1228   The prawn boat sank near Newcastle 

           -0.0124   Hi

           0.0126   Now I am leaning to drive a car.

           0.3637   It is dangerous to ride in the dark.

           0.6278   A 12-foot boat capsized near the harbor breakwater 

           0.3586   The boat was heading from Latvia to Sweden when it sank near Copenhagen 

           0.0383   I do not have a sailboat

           0.7389   A fishing boat ran aground on a sandbank early yesterday near Aberdeen 

           0.3809   We can not get there by train.

           0.6589   A sailing boat is fun to sail

           0.0253   I usually take the bus.

           0.3473   A dinghy is cheap

           0.4126   The boat ran aground on the way from Latvia to Sweden 

           0.4956   My friend goes on long hikes every month.

           0.5151   The sailing boat was swept on to rocks off the Corsican coast 

           0.333   I do not like to ride in the rain.

Planes are faster than trains.

           -0.0057   The wind drove two boats aground off Santa Island 

           0

           0.0329   Sailing is great fun

           -0.0057   The fishing boat ran aground on San Clemente Island 

           0.4254   I want to visit my uncle on the farm.
           0.3169   I have never walked that far.

           0.1073   They sailed a fishing boat to Dry Tortugas 

           0.3441   I sail sometimes in the Summer

           0.0706   He owns a tractor and lets me drive it.

           0.0262   The yacht ran aground on the North coast of Caithness 

           -0.0238   The French yacht was capsized by a strong squall

           0.293   Now I am leaning to drive a car.

           0.0418   It is dangerous to ride in the dark.

           -0.008   A 12-foot boat capsized near the harbor breakwater 

           0.0498   The boat was heading from Latvia to Sweden when it sank near Copenhagen 

           0.3658   I do not have a sailboat

           -0.0057   A fishing boat ran aground on a sandbank early yesterday near Aberdeen 

           0.1287   We can not 

           0.002   Now I am leaning to drive a car.

           0.3463   It is dangerous to ride in the dark.

           0.3183   A 12-foot boat capsized near the harbor breakwater 

           0.3432   The boat was heading from Latvia to Sweden when it sank near Copenhagen 

           -0.0034   I do not have a sailboat

           0.3158   A fishing boat ran aground on a sandbank early yesterday near Aberdeen 

           0.3319   We can not get there by train.

           0.6615   A sailing boat is fun to sail

           0.0049   I usually take the bus.

           0.3352   A dinghy is cheap

           -0.0231   The boat ran aground on the way from Latvia to Sweden 

           0.3567   My friend goes on long hikes every month.

           0.4961   The sailing boat was swept on to rocks off the Corsican coast 

           0.355   I do not like to ride in the rain.

The prawn boat sank near Newcastle 

           0.1228   The wind drove two boats aground off Santa Island 

       

           0.3419   My friend goes on long hikes every month.

           0.5064   The sailing boat was swept on to rocks off the Corsican coast 

           0.392   I do not like to ride in the rain.

The fishing boat ran aground on San Clemente Island 

           0.2124   The wind drove two boats aground off Santa Island 

           0.5088   Riding in the rain is no fun.

           0.3526   A yacht is expensive

           0.3987   It is too far to walk.

           0.3982   A boat skipper let a barge run aground in January off Puerto Rico 

           0.7389   A Navy patrol boat went aground on an uncharted sandbar 

           0.3536   Planes are faster than trains.

           0.0108   I would like a sailboat

           0.3221   A sloop is affordable

           -0.0057   I already know how to ride the bicycle.

           1.0   The boat ran aground on a wreck on its way back 

           0.0534   Seven crew abandoned the boat when it struck a rock 

           0.0648   In fou

           0.0247   My friend goes on long hikes every month.

           0.0273   The sailing boat was swept on to rocks off the Corsican coast 

           0.0672   I do not like to ride in the rain.

I sail sometimes in the Summer

           0.0405   The wind drove two boats aground off Santa Island 

           0.5035   Riding in the rain is no fun.

           0.3385   A yacht is expensive

           0.3682   It is too far to walk.

           0.358   A boat skipper let a barge run aground in January off Puerto Rico 

           0.3472   A Navy patrol boat went aground on an uncharted sandbar 

           0.3444   Planes are faster than trains.

           0.3474   I would like a sailboat

           0.3156   A sloop is affordable

           0.3441   I already know how to ride the bicycle.

           0.3678   The boat ran aground on a wreck on its way back 

           0.045   Seven crew abandoned the boat when it struck a rock 

           0.1165   In fourteen hundred and nin

           0.0438   The wind drove two boats aground off Santa Island 

           -0.02   Riding in the rain is no fun.

           -0.0033   A yacht is expensive

           -0.003   It is too far to walk.

           0.001   A boat skipper let a barge run aground in January off Puerto Rico 

           0.0126   A Navy patrol boat went aground on an uncharted sandbar 

           -0.0116   Planes are faster than trains.

           0.3395   I would like a sailboat

           0.0117   A sloop is affordable

           0.293   I already know how to ride the bicycle.

           0.0092   The boat ran aground on a wreck on its way back 

           0.0572   Seven crew abandoned the boat when it struck a rock 

           0.0283   In fourteen hundred and ninety two, Columbus sailed the ocean blue.

           0.002   Powerboats are more fun than sailboats

           -0.0072   The prawn boat sank near Newcastle 

           -0.0181   His boat was broken in strong winds North-East of Lerw

           0.4011   I usually take the bus.

           0.0022   A dinghy is cheap

           0.0317   The boat ran aground on the way from Latvia to Sweden 

           0.0215   My friend goes on long hikes every month.

           -0.0017   The sailing boat was swept on to rocks off the Corsican coast 

           0.3775   I do not like to ride in the rain.

A fishing boat ran aground on a sandbank early yesterday near Aberdeen 

           0.2124   The wind drove two boats aground off Santa Island 

           0.5088   Riding in the rain is no fun.

           0.3526   A yacht is expensive

           0.3987   It is too far to walk.

           0.3982   A boat skipper let a barge run aground in January off Puerto Rico 

           0.7389   A Navy patrol boat went aground on an uncharted sandbar 

           0.3536   Planes are faster than trains.

           0.0108   I would like a sailboat

           0.3221   A sloop is affordable

           -0.0057   I already know how to ride 

           0.3211   Sailing is great fun

           0.3797   The fishing boat ran aground on San Clemente Island 

           0.0001   I want to visit my uncle on the farm.
           0.3529   I have never walked that far.

           0.0317   They sailed a fishing boat to Dry Tortugas 

           0.3262   I sail sometimes in the Summer

           -0.0091   He owns a tractor and lets me drive it.

           0.3759   The yacht ran aground on the North coast of Caithness 

           0.541   The French yacht was capsized by a strong squall

           -0.0348   Now I am leaning to drive a car.

           0.3525   It is dangerous to ride in the dark.

           0.3675   A 12-foot boat capsized near the harbor breakwater 

           0.3136   The boat was heading from Latvia to Sweden when it sank near Copenhagen 

           0.0022   I do not have a sailboat

           0.3797   A fishing boat ran aground on a sandbank early yesterday near Aberdeen 

           0.3897   We can not g

## UNIT TEST EMBRYO


In [ ]:
dud=newrandomvector(dimensionality,200) #subject

foo=newrandomvector(dimensionality,200) #object
bar=modify(foo,0.5) #another object

hey=newrandomvector(dimensionality,500) #verb
ho=modify(hey,0.5) #another verb

vectorspace["dud"]=dud
vectorspace["foo"]=foo
vectorspace["bar"]=bar
vectorspace["hey"]=hey
vectorspace["ho"]=ho
# to use the above now indexspace vectors need to be added
indexspace["dud"]=vectorspace["dud"]
indexspace["foo"]=vectorspace["foo"]
indexspace["bar"]=vectorspace["bar"]
indexspace["hey"]=vectorspace["hey"]
indexspace["ho"]=vectorspace["ho"]


utterances1=["I saw a cat.","Cat saw a cat.","I saw the cat."] # which is or should be closer to #1, #2 or #3?
utterances2=["I feed a cat.", "I feed the cat.","I saw a cat."] # ditto
utterances3=["dud hey a bar","dud hey a foo","dud ho the bar","dud hey the bar"]

utterances=utterances2
utterancespace1={}
utterancespace2={}
utterancespace3={}
utterancespace4={}
utterancespace5={}
utterancespace6={}
for u in utterances:
    struct=semanticparse(u)
    utterancespace1[u] = utterancevector(struct,True,False,False) #only lex
    utterancespace2[u] = utterancevector(struct,False,True,False) # only morph
    utterancespace3[u] = utterancevector(struct,True,True,False)
    utterancespace4[u] = utterancevector(struct,True,False,True)
    utterancespace5[u] = utterancevector(struct,False,True,True)
    utterancespace6[u] = utterancevector(struct,True,True,True)
for u in utterances:
    print(u)
    for v in utterances:
        print(
            sparsecosine(utterancespace1[u],utterancespace1[v]),"\t",
              sparsecosine(utterancespace2[u],utterancespace2[v]),"\t",
#              sparsecosine(utterancespace3[u],utterancespace3[v]),"\t",
#              sparsecosine(utterancespace4[u],utterancespace4[v]),"\t",
#              sparsecosine(utterancespace5[u],utterancespace5[v]),"\t",
              sparsecosine(utterancespace6[u],utterancespace6[v]),"\t",
              v)
for u in utterances:
    print(u)
    print(
        "Indef",
        sparsecosine(utterancespace2[u],permute(patientoperator,indefinite)),"\t",
        sparsecosine(utterancespace6[u],permute(patientoperator,indefinite)) )
    print(
        "Def",
        sparsecosine(utterancespace2[u],permute(patientoperator,definite)),"\t",
        sparsecosine(utterancespace6[u],permute(patientoperator,definite)) )
        


In [ ]:
def dofdgfile(fdgfilename):
    utterances=[]
    gameon=False
    taglinepattern = re.compile("^<([\/\w]+)>\s*$")
    sgmllinepattern = re.compile("^\d+\s+<(\w+)>\s+<\w+>\s*$")
    wordlinepattern = re.compile("^(\d+)\s+([\d\/\'\w]+)\s+([\d\/\w]+)\s+(\w+):>(\d+)\s+(.*)$")
    fdgfile=open(fdgfilename,'r')
    a=0
    b=0
    u=0
    w=0

    fdgline=fdgfile.readline()
    notes={}
    tokens=[]
    itemindex={}
    utterance=clause("empty")
    agent=None
    patient=None
    verb=None
    surfacestring=""
    while fdgline:
        a+=1
        tag=taglinepattern.match(fdgline)
        eos=sgmllinepattern.match(fdgline)
        word=wordlinepattern.match(fdgline)
        if tag:
#            print(">",m.groups()[0],"<")
            if (tag.groups()[0] == "HEADLINE"  or tag.groups()[0] == "TEXT" ):
                gameon=True
            if (tag.groups()[0] == "/HEADLINE" or tag.groups()[0] == "/TEXT" ):
                gameon=False
        elif eos:
            u+=1
#            if (len(tokens) > 1):
#                print(tokens)

#            if not utterance is None:
            if len(tokens) > 0:
                utterance.surfacestring=surfacestring 
                utterance.agent=agent
                utterance.patient=patient
                utterance.verb=verb
                utterance.tokens=tokens
                utterances.append(utterance)
            utterance=clause("empty")
            tokens=[]
            itemindex={}
            notes={}
            agent=None
            patient=None
            verb=None
            surfacestring=""

        elif word:
            if gameon:
                w+=1
                note=""
                lemma=word.groups()[2]
                surface=word.groups()[1]
                if (lemma in indexspace):
                    pass
                else:
                    indexspace[lemma] = newrandomvector(dimensionality,denseness)
                    vectorspace[lemma] = {}
                if (surface in indexspace):
                    pass
                else:
                    indexspace[surface] = newrandomvector(dimensionality,denseness)
                    vectorspace[surface] = {}
                position=word.groups()[0]
                role=word.groups()[3]
                head=word.groups()[4]
                morph=word.groups()[5]
                itemindex[position] = lemma
                if position in notes:
                    note=notes[position]
                if role=="det":
                    if surface=="a" or surface=="an":
                        notes[head]="INDEFINITE"
                    else:
                        notes[head]="DEFINITE"
                elif role=="main":
                    verb=event(lemma)
                    tokens.append(lemma)
                    utterance.event = verb
                    utterance.event.tense=="PRESENT"
                elif role=="subj":
                    notes[position]="AGENT" # for now
                    agent=referent(surface)
                    if position in notes:
                        if notes[position]=="DEFINITE":
                            agent.definite=True
                        elif notes[position]=="INDEFINITE":
                            agent.definite=False
                    else:
                        notes[head]="DEFINITE"
                    tokens.append(lemma)
                elif role=="obj":
                    notes[position]="PATIENT" # for now
                    agent=referent(surface)
                    if position in notes:
                        if notes[position]=="DEFINITE":
                            agent.definite=True
                        elif notes[position]=="INDEFINITE":
                            agent.definite=False
                    else:
                        notes[head]="DEFINITE"
                    tokens.append(lemma)
                else:
                    tokens.append(lemma)
        else:
            b+=1
            if gameon:
                pass #print("dofdgfile(): strange line: ",fdgline)
#        print(a," ",u," ",w," ",b)
        try:
            fdgline=fdgfile.readline()
        except:
            print("dofdgfile(): broken line at line "+str(a))
            a += 1
            fdgline=fdgfile.readline()
    print(a," lines seen; ",u," utterances seen ",w," words seen",b," items disregarded")
    fdgfile.close()
    return utterances

#### def utterancevectordebug(v1,utterance,lexical=True,morphology=True,syntacticroles=True):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    print(sparsecosine(v1,uttvec))
    if lexical:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.patient.surfacestring])))#,sparselength(vectorspace[utterance.patient]))
        if (utterance.event):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.event.surfacestring])))#,sparselength(vectorspace[utterance.verb]))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.agent.surfacestring])))#,sparselength(vectorspace[utterance.agent]))
    print(sparsecosine(v1,uttvec))
    if morphology:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],present)) )  
    print(sparsecosine(v1,uttvec))
    if False:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(patientoperator,definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(patientoperator,indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,present)) )  
    print(sparsecosine(v1,uttvec))
    if syntacticroles:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,sparsemultiply(patientoperator,normalise(indexspace[utterance.patient.surfacestring])))
        if (utterance.event):
            uttvec=sparseadd(uttvec,sparsemultiply(mainverboperator,normalise(indexspace[utterance.event.surfacestring])))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,sparsemultiply(agentoperator,normalise(indexspace[utterance.agent.surfacestring])))
    print(sparsecosine(v1,uttvec))
    return uttvec


In [34]:
vs={}
nb={}
for u in utterances:
    vs[u]=utterancevector(u)
for v in vs:
    max=0
    neighbour=None
    for w in vs:
        if v==w:
            continue
        vwc=sparsecosine(vs[v],vs[w])
        if vwc > max:
            max = vwc
            neighbour=w
    print(v.tokens)
    if neighbour:
        print("\t",max,"\t",neighbour.tokens)
        nb[v]=neighbour

NameError: name 'utterances' is not defined

In [ ]:
us=dofdgfile(fdgtextfile)
for u in us:
    print(u.tokens)
    